# Viser + MPlib Demo: Robot Visualization and IK/FK

This notebook demonstrates:
- Setting up a Viser visualization server
- Loading and visualizing robots from URDF files
- Using MPlib for inverse kinematics (IK) and forward kinematics (FK)
- Visualizing robot trajectories
- Helper functions for IK/FK calculations


## 1. Setup and Imports


In [1]:
import numpy as np
import time
from pathlib import Path

import viser
from viser.extras import ViserUrdf
import mplib

# Initialize the Viser server (opens a web interface at http://localhost:8080)
server = viser.ViserServer()


╭────── viser (listening *:8080) ───────╮
│             ╷                         │
│   HTTP      │ http://localhost:8080   │
│   Websocket │ ws://localhost:8080     │
│             ╵                         │
╰───────────────────────────────────────╯

(viser) Connection opened (0, 1 total), 5 persistent messages

## 2. Scene Setup

Add a world frame and configure the visualization scene.


In [2]:
# Add world frame for reference
server.scene.add_frame(
    "/world",
    wxyz=(1.0, 0.0, 0.0, 0.0),  # Quaternion (w, x, y, z) - identity rotation
    position=(0, 0, 0),
    axes_length=0.2,
    axes_radius=0.01,
)


FrameHandle(show_axes=True, axes_length=0.2, axes_radius=0.01, origin_radius=0.02, origin_color=(236, 236, 0))

## 3. Load Robot Models

Load the robot URDF and create visualization instances. Here we set up two robot instances side-by-side.


In [3]:
# Path to URDF file (adjust if needed)
URDF_PATH = Path("wxai_follower.urdf")
SRDF_PATH = Path("wxai_follower.srdf")  # Optional: for collision checking

# Create base frames for two robots side-by-side
Y_OFFSET = 0.28  # Separation between robots
base_a = server.scene.add_frame("/robots/a_base", position=(0.0, +Y_OFFSET, 0.0), show_axes=False)
base_b = server.scene.add_frame("/robots/b_base", position=(0.0, -Y_OFFSET, 0.0), show_axes=False)

# Load robot A
robot_a = ViserUrdf(
    server,
    urdf_or_path=URDF_PATH,
    load_meshes=True,
    load_collision_meshes=True,
    collision_mesh_color_override=(0.2, 0.2, 0.2, 0.5),  # Semi-transparent gray
    root_node_name="/robots/a_base",
)

# Load robot B
robot_b = ViserUrdf(
    server,
    urdf_or_path=URDF_PATH,
    load_meshes=True,
    load_collision_meshes=True,
    collision_mesh_color_override=(0.2, 0.2, 0.2, 0.5),
    root_node_name="/robots/b_base",
)

# Initialize both robots to home position (8-DoF: 6 joints + 2 gripper)
home_config = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
robot_a.update_cfg(home_config)
robot_b.update_cfg(home_config)

print("✓ Robots loaded and initialized")


✓ Robots loaded and initialized


## 4. Initialize MPlib Planner

Set up the motion planning library for IK/FK calculations.


In [4]:
# Initialize MPlib planner
planner = mplib.Planner(
    urdf=str(URDF_PATH),
    srdf=str(SRDF_PATH) if SRDF_PATH.exists() else None,  # Optional SRDF for collision checking
    move_group="ee_gripper_link",  # End-effector link name
)

print("✓ MPlib planner initialized")


✓ MPlib planner initialized


## 5. Helper Functions: Forward and Inverse Kinematics

Utility functions for easy IK/FK calculations.


## 5b. Collision-Aware Helper Functions

Additional functions for collision checking and collision-free IK.


In [ ]:
def check_collision(q, robot_model=None):
    """
    Check if a joint configuration results in self-collision.
    
    Args:
        q: Joint configuration (can be move-group size or full DoF)
        robot_model: Robot model (uses planner.robot if None)
        
    Returns:
        True if collision-free, False if collision detected
    """
    if robot_model is None:
        robot_model = planner.robot
    
    # Pad to full DoF if needed
    q_full = planner.pad_move_group_qpos(np.asarray(q, float))
    
    # Set the configuration
    robot_model.set_qpos(q_full)
    
    # Check for collisions using the robot model's collision checker
    try:
        # Try the standard collision checking method
        is_valid = robot_model.is_valid()
        return is_valid
    except AttributeError:
        # Fallback: try accessing pinocchio model directly
        try:
            pin = robot_model.get_pinocchio_model()
            # Compute collisions
            pin.compute_collisions(q_full)
            # Check if any collisions detected
            collision_results = pin.get_collision_results()
            return not any(collision_results)
        except:
            # If collision checking not available, assume valid
            print("⚠ Warning: Collision checking not available, assuming valid")
            return True


def ee_ik_collision_free(pos, quat=None, start=None, tol=1e-2, ee=EE_LINK, check_collisions=True):
    """
    Inverse kinematics with collision checking:
    - Tries Planner.IK (exact if reachable, else approximate if tolerance allows)
    - Filters solutions to only return collision-free configurations
    - Falls back to CLIK if no collision-free IK solution found
    
    Args:
        pos: Target position [x, y, z]
        quat: Target quaternion [w, x, y, z] (optional, uses current orientation if None)
        start: Starting joint configuration (optional)
        tol: Position tolerance
        ee: End-effector link name
        check_collisions: If True, only return collision-free solutions
        
    Returns:
        Full qpos suitable for viewers/FK (collision-free if check_collisions=True)
    """
    pos = np.asarray(pos, float).tolist()
    
    # Use current configuration if start not provided
    if start is None:
        start = planner.robot.get_qpos()
    
    # Use current orientation if quaternion not provided (position-priority IK)
    if quat is None:
        _, quat_now = ee_fk(start, ee)
        quat = quat_now
    quat = np.asarray(quat, float).tolist()

    goal = mplib.Pose(pos, quat)

    # Try Planner.IK (handles different API versions)
    try:
        status, q_goals = planner.IK(
            goal_pose=goal, 
            start_qpos=np.asarray(start, float),
            threshold=tol, 
            return_closest=True
        )
    except TypeError:
        # Older API (no return_closest)
        status, q_goals = planner.IK(
            goal_pose=goal, 
            start_qpos=np.asarray(start, float),
            threshold=tol
        )

    # Handle multiple solutions - check for collision-free ones
    if status == "Success" and q_goals is not None:
        # Handle both single solution and multiple solutions
        if isinstance(q_goals, np.ndarray) and len(q_goals.shape) == 1:
            # Single solution
            solutions = [q_goals]
        elif isinstance(q_goals, list):
            # List of solutions
            solutions = q_goals
        else:
            # Try to extract solutions
            solutions = [q_goals] if not isinstance(q_goals, np.ndarray) else [q_goals]
        
        # Check each solution for collisions
        collision_free_found = False
        for q_sol_group in solutions:
            q_full = planner.pad_move_group_qpos(q_sol_group)
            
            if check_collisions:
                if check_collision(q_full):
                    print(f"✓ Found collision-free IK solution (checked {len(solutions)} solution(s))")
                    return q_full  # Return first collision-free solution
            else:
                return q_full  # Return first solution without checking
        
        if check_collisions and not collision_free_found:
            print(f"⚠ All {len(solutions)} IK solution(s) have collisions, trying CLIK...")

    # If no collision-free IK solution found, try CLIK
    pin = planner.robot.get_pinocchio_model()
    ee_idx = pin.get_link_names().index(ee)
    q_full0 = planner.pad_move_group_qpos(np.asarray(start, float))
    q_full_best, _, _ = pin.compute_IK_CLIK(ee_idx, goal, q_full0)
    
    # Check collision for CLIK solution
    if check_collisions:
        if check_collision(q_full_best):
            print("✓ CLIK solution is collision-free")
        else:
            print("⚠ Warning: CLIK solution has collisions, returning anyway")
    
    return q_full_best

print("✓ Collision-aware helper functions defined")


In [5]:
EE_LINK = "ee_gripper_link"  # End-effector link name

def ee_fk(q, ee=EE_LINK):
    """
    Forward kinematics: joint angles -> end-effector pose.
    
    Args:
        q: Joint angles (can be move-group size or full DoF)
        ee: End-effector link name
        
    Returns:
        position: (3,) array [x, y, z]
        quaternion: (4,) array [w, x, y, z]
    """
    pin = planner.robot.get_pinocchio_model()
    q_full = planner.pad_move_group_qpos(np.asarray(q, float))  # Pad to full DoF if needed
    pin.compute_forward_kinematics(q_full)
    ee_idx = pin.get_link_names().index(ee)
    pose = pin.get_link_pose(ee_idx)
    return np.asarray(pose.get_p(), float), np.asarray(pose.get_q(), float)


def ee_ik(pos, quat=None, start=None, tol=1e-2, ee=EE_LINK):
    """
    Inverse kinematics (best-effort):
    - Tries Planner.IK (exact if reachable, else approximate if tolerance allows)
    - Falls back to CLIK if no IK solution found
    
    Args:
        pos: Target position [x, y, z]
        quat: Target quaternion [w, x, y, z] (optional, uses current orientation if None)
        start: Starting joint configuration (optional)
        tol: Position tolerance
        ee: End-effector link name
        
    Returns:
        Full qpos suitable for viewers/FK
    """
    pos = np.asarray(pos, float).tolist()
    
    # Use current configuration if start not provided
    if start is None:
        start = planner.robot.get_qpos()
    
    # Use current orientation if quaternion not provided (position-priority IK)
    if quat is None:
        _, quat_now = ee_fk(start, ee)
        quat = quat_now
    quat = np.asarray(quat, float).tolist()

    goal = mplib.Pose(pos, quat)

    # Try Planner.IK (handles different API versions)
    try:
        status, q_goals = planner.IK(
            goal_pose=goal, 
            start_qpos=np.asarray(start, float),
            threshold=tol, 
            return_closest=True
        )
    except TypeError:
        # Older API (no return_closest)
        status, q_goals = planner.IK(
            goal_pose=goal, 
            start_qpos=np.asarray(start, float),
            threshold=tol
        )

    q_sol_group = None
    if status == "Success" and q_goals is not None:
        q_sol_group = (q_goals if isinstance(q_goals, np.ndarray) else q_goals[0])

    if q_sol_group is not None:
        return planner.pad_move_group_qpos(q_sol_group)  # Return full DoF

    # Fallback: CLIK (best-effort closest pose, even if unreachable)
    pin = planner.robot.get_pinocchio_model()
    ee_idx = pin.get_link_names().index(ee)
    q_full0 = planner.pad_move_group_qpos(np.asarray(start, float))
    q_full_best, _, _ = pin.compute_IK_CLIK(ee_idx, goal, q_full0)
    return q_full_best

print("✓ Helper functions defined")


✓ Helper functions defined


## 6. Example: Forward Kinematics

Compute the end-effector position and orientation from joint angles.


In [6]:
# Example joint angles (in degrees, converted to radians)
joint_angles_deg = [-38.28, 23.20, 7.99, 22.87, -33.76, 26.68]
joint_angles_rad = np.radians(joint_angles_deg)

# Compute forward kinematics
position, quaternion = ee_fk(joint_angles_rad)

print(f"Joint angles (deg): {joint_angles_deg}")
print(f"End-effector position: {position}")
print(f"End-effector quaternion (w,x,y,z): {quaternion}")

# Update robot visualization
robot_a.update_cfg(np.append(joint_angles_rad, [0.0, 0.0]))  # Add gripper DoF

# Visualize end-effector position (green sphere)
server.scene.add_icosphere(
    "/end_effector",
    radius=0.015,
    color=(0.0, 1.0, 0.0),  # Green
    position=(position[0], position[1] + Y_OFFSET, position[2]),
)


Joint angles (deg): [-38.28, 23.2, 7.99, 22.87, -33.76, 26.68]
End-effector position: [ 0.25718745 -0.07345394  0.230426  ]
End-effector quaternion (w,x,y,z): [ 0.97915298  0.19181383 -0.06164625 -0.02581921]


IcosphereHandle(radius=0.015, subdivisions=3, color=(0, 255, 0), wireframe=False, opacity=None, flat_shading=False, side='front', material='standard', cast_shadow=True, receive_shadow=True)

## 7. Example: Inverse Kinematics

Find joint angles to reach a target end-effector pose.


In [7]:
# Set initial configuration
initial_config = np.array([0.0, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0])
robot_a.update_cfg(initial_config)
robot_b.update_cfg(initial_config)

# Define target position
target_position = np.array([0.4, 0.25, 0.3])
target_quaternion = [1.0, 0.0, 0.0, 0.0]  # Identity quaternion (w, x, y, z)

# Visualize target (red sphere)
server.scene.add_icosphere(
    "/target",
    radius=0.02,
    color=(1.0, 0.0, 0.0),  # Red
    position=(target_position[0], target_position[1] + Y_OFFSET, target_position[2]),
)
print(f"✓ Target position: {target_position}")

# Solve inverse kinematics
q_solution = ee_ik(
    pos=target_position,
    quat=target_quaternion,
    start=initial_config
)

# Update robot to IK solution
robot_a.update_cfg(q_solution)
print(f"✓ IK solution: {q_solution[:6]} (first 6 joints)")

# Verify with forward kinematics
actual_position, _ = ee_fk(q_solution[:6])
error = np.linalg.norm(actual_position - target_position)
print(f"✓ Position error: {error*1000:.2f} mm")

# Visualize actual end-effector position (green sphere)
server.scene.add_icosphere(
    "/end_effector_ik",
    radius=0.015,
    color=(0.0, 1.0, 0.0),  # Green
    position=(actual_position[0], actual_position[1] + Y_OFFSET, actual_position[2]),
)


✓ Target position: [0.4  0.25 0.3 ]
✓ IK solution: [ 8.55824475e-01  1.57183831e+00  1.06076054e+00  5.11077777e-01
  8.55824475e-01 -1.16326330e-16] (first 6 joints)
✓ Position error: 0.01 mm


IcosphereHandle(radius=0.015, subdivisions=3, color=(0, 255, 0), wireframe=False, opacity=None, flat_shading=False, side='front', material='standard', cast_shadow=True, receive_shadow=True)

## 8. IK Examples: Reachable vs Unreachable Positions

Demonstrate IK with a reachable position (exact solution) and an unreachable position (CLIK fallback).


## 8b. IK with Collision Avoidance

Demonstrate IK that avoids self-collisions by checking all solutions and selecting collision-free ones.


In [ ]:
# Set initial configuration
start_config = np.array([0.0, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0])
robot_a.update_cfg(start_config)

print("=" * 60)
print("IK with Collision Avoidance")
print("=" * 60)

# Define target position
target_position = np.array([0.4, 0.25, 0.3])
target_quaternion = [1.0, 0.0, 0.0, 0.0]

# Visualize target (red sphere)
server.scene.add_icosphere(
    "/target_collision_free",
    radius=0.02,
    color=(1.0, 0.0, 0.0),  # Red
    position=(target_position[0], target_position[1] + Y_OFFSET, target_position[2]),
)
print(f"Target position: {target_position}")

# Solve IK with collision checking enabled
print("\nSolving IK with collision checking...")
q_collision_free = ee_ik_collision_free(
    pos=target_position,
    quat=target_quaternion,
    start=start_config,
    check_collisions=True
)

# Update robot to collision-free IK solution
robot_a.update_cfg(q_collision_free)

# Verify with forward kinematics
actual_pos, _ = ee_fk(q_collision_free[:6])
error = np.linalg.norm(actual_pos - target_position)

print(f"\n✓ IK solution: {q_collision_free[:6]} (first 6 joints)")
print(f"✓ Position error: {error*1000:.2f} mm")

# Check collision status
is_collision_free = check_collision(q_collision_free)
print(f"✓ Collision status: {'Collision-free ✓' if is_collision_free else 'Collision detected ✗'}")

# Visualize actual end-effector position (green sphere)
server.scene.add_icosphere(
    "/ee_collision_free",
    radius=0.015,
    color=(0.0, 1.0, 0.0),  # Green
    position=(actual_pos[0], actual_pos[1] + Y_OFFSET, actual_pos[2]),
)

print("\n" + "=" * 60)
print("Comparison: IK without collision checking")
print("=" * 60)

# Compare with IK without collision checking
q_no_check = ee_ik(
    pos=target_position,
    quat=target_quaternion,
    start=start_config
)

is_collision_free_no_check = check_collision(q_no_check)
print(f"Standard IK collision status: {'Collision-free ✓' if is_collision_free_no_check else 'Collision detected ✗'}")

if not is_collision_free_no_check:
    print("⚠ Standard IK solution has collisions!")
    print("  → Collision-aware IK ensures safe configurations")
else:
    print("✓ Standard IK solution is also collision-free in this case")

print("\n" + "=" * 60)


In [11]:
# Set initial configuration
import time
start_config = np.array([0.0, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0])
robot_a.update_cfg(start_config)

print("=" * 60)
print("Example 1: IK with REACHABLE position")
print("=" * 60)

# Reachable target position (within workspace)
reachable_target = np.array([0.4, 0.25, 0.3])
target_quat = [1.0, 0.0, 0.0, 0.0]

# Visualize target (red sphere)
server.scene.add_icosphere(
    "/target_reachable",
    radius=0.02,
    color=(1.0, 0.0, 0.0),  # Red
    position=(reachable_target[0], reachable_target[1] + Y_OFFSET, reachable_target[2]),
)

# Solve IK
q_reachable = ee_ik(pos=reachable_target, quat=target_quat, start=start_config)
robot_a.update_cfg(q_reachable)

# Verify with FK
actual_pos, _ = ee_fk(q_reachable[:6])
error = np.linalg.norm(actual_pos - reachable_target)

print(f"Target: {reachable_target}")
print(f"Actual: {actual_pos}")
print(f"Position error: {error*1000:.2f} mm")
print("✓ IK solution found (exact or approximate within tolerance)")

# Visualize actual end-effector (green sphere)
server.scene.add_icosphere(
    "/ee_reachable",
    radius=0.015,
    color=(0.0, 1.0, 0.0),  # Green
    position=(actual_pos[0], actual_pos[1] + Y_OFFSET, actual_pos[2]),
)

print("\n" + "=" * 60)
print("Example 2: IK with UNREACHABLE position (CLIK fallback)")
print("=" * 60)
time.sleep(5)
# Unreachable target position (outside workspace)
unreachable_target = np.array([0.0, 0.0, 0.0])  # Too far from base

# Visualize target (orange sphere to indicate unreachable)
server.scene.add_icosphere(
    "/target_unreachable",
    radius=0.02,
    color=(1.0, 0.5, 0.0),  # Orange
    position=(unreachable_target[0], unreachable_target[1] + Y_OFFSET, unreachable_target[2]),
)

# Try IK - will fall back to CLIK if unreachable
q_unreachable = ee_ik(pos=unreachable_target, quat=target_quat, start=start_config)
robot_a.update_cfg(q_unreachable)

# Verify with FK
actual_pos_unreachable, _ = ee_fk(q_unreachable[:6])
error_unreachable = np.linalg.norm(actual_pos_unreachable - unreachable_target)

print(f"Target: {unreachable_target}")
print(f"Actual: {actual_pos_unreachable}")
print(f"Position error: {error_unreachable*1000:.2f} mm")
print("⚠ CLIK used (best-effort solution for unreachable target)")

# Visualize actual end-effector (yellow sphere to indicate approximation)
server.scene.add_icosphere(
    "/ee_unreachable",
    radius=0.015,
    color=(1.0, 1.0, 0.0),  # Yellow
    position=(actual_pos_unreachable[0], actual_pos_unreachable[1] + Y_OFFSET, actual_pos_unreachable[2]),
)

print("\n" + "=" * 60)
print("Summary:")
print("  - Red sphere: Reachable target")
print("  - Orange sphere: Unreachable target")
print("  - Green sphere: Actual EE position (reachable case)")
print("  - Yellow sphere: Actual EE position (unreachable case, CLIK)")
print("=" * 60)


Example 1: IK with REACHABLE position
Target: [0.4  0.25 0.3 ]
Actual: [0.39999528 0.24999144 0.29999912]
Position error: 0.01 mm
✓ IK solution found (exact or approximate within tolerance)

Example 2: IK with UNREACHABLE position (CLIK fallback)
Target: [0. 0. 0.]
Actual: [7.83010764e-06 0.00000000e+00 5.85773878e-06]
Position error: 0.01 mm
⚠ CLIK used (best-effort solution for unreachable target)

Summary:
  - Red sphere: Reachable target
  - Orange sphere: Unreachable target
  - Green sphere: Actual EE position (reachable case)
  - Yellow sphere: Actual EE position (unreachable case, CLIK)


## 9. Example: Visualize Trajectory from Data

Load trajectory data and visualize the end-effector path. 

**Note:** This example requires a parquet file with trajectory data. Uncomment and adjust the file path as needed.


In [ ]:
# Uncomment and adjust the path to your trajectory data file
# import pandas as pd
# 
# TRAJECTORY_FILE = "path/to/your/episode_000000.parquet"
# df = pd.read_parquet(TRAJECTORY_FILE)
# 
# # Extract trajectory points using forward kinematics
# trajectory_points = []
# for idx in range(df.shape[0]):
#     angles = df["observation.state"][idx]
#     # Assuming first 6 values are joint angles in degrees
#     joint_angles = np.radians(angles[:6])
#     position, _ = ee_fk(joint_angles)
#     # Apply Y offset for visualization (robot A)
#     trajectory_points.append([position[0], position[1] + Y_OFFSET, position[2]])
# 
# # Visualize trajectory as spline segments
# for i in range(len(trajectory_points) - 1):
#     server.scene.add_spline_catmull_rom(
#         f"traj_segment_{i}",
#         positions=np.array([trajectory_points[i], trajectory_points[i+1]]),
#         color=(0.5, 0.5, 1.0),  # Light blue
#         segments=2,
#     )
# 
# print(f"✓ Visualized {len(trajectory_points)} trajectory points")

print("Trajectory visualization example (commented out - uncomment and provide data file)")


## 10. Example: Animate Robot from Trajectory Data

Animate robots using trajectory data from a file.

**Note:** Uncomment and adjust the file path as needed.


In [ ]:
# Uncomment and adjust the path to your trajectory data file
# import pandas as pd
# 
# TRAJECTORY_FILE = "path/to/your/episode_000000.parquet"
# df = pd.read_parquet(TRAJECTORY_FILE)
# 
# # Animate robots through trajectory
# FRAME_RATE = 20  # frames per second
# 
# for idx in range(df.shape[0]):
#     angles = df["observation.state"][idx]
#     
#     # Extract joint angles for robot A (first 6 joints)
#     joint_angles_a = np.radians(angles[:6])
#     # Extract gripper values (adjust indices based on your data format)
#     gripper_a = (angles[6] / 2) / 10000  # Example conversion
#     config_a = np.append(joint_angles_a, [gripper_a, gripper_a])
#     
#     # Extract joint angles for robot B (next 6 joints)
#     joint_angles_b = np.radians(angles[7:13])
#     gripper_b = (angles[13] / 2) / 10000
#     config_b = np.append(joint_angles_b, [gripper_b, gripper_b])
#     
#     # Update robot configurations
#     robot_a.update_cfg(config_a)
#     robot_b.update_cfg(config_b)
#     
#     time.sleep(1.0 / FRAME_RATE)
# 
# print("✓ Animation complete")

print("Animation example (commented out - uncomment and provide data file)")
